### Rectangular dataset : Raw_Recipe.csv

The dataset contains raw details about recipes mainly contains following **important** details.


name	
id	
minutes	
tags	
nutrition	
n_steps	steps	
description	
ingredients	
n_ingredients


In [8]:
import pandas as pd

class rawRecipe:
    def __init__(self):
        self.file_path = './Dataset/RAW_recipes.csv'
        self.data = None
    
    ###Loads the dataset from a CSV file.
    def load_data(self):
        self.data = pd.read_csv(self.file_path)
        print(f"Data loaded successfully. Shape: {self.data.shape}")
        return self.data
    
    def summarize_dataset(self):
        """Provides basic statistics of the dataset."""
        if self.data is not None:
            minutesSummary = self.data['minutes'].describe()
            stepsSummary = self.data['n_steps'].describe()
            ingrediantsSummary = self.data['n_ingredients'].describe()
            print("Data summary: Time ( minutes )")
            print(minutesSummary)
            print("Data summary: Number of Steps")
            print(stepsSummary)
            print("Data summary: Required Ingrediants")
            print(ingrediantsSummary)
        else:
            print("Unable to load Recepe data.")
    
    # def check_null_values(self):
    #     """Checks for missing values in the dataset."""
    #     if self.data is not None:
    #         nulls = self.data.isnull().sum()
    #         print("Missing values in the dataset:")
    #         print(nulls)
    #         return nulls
    #     else:
    #         print("Data not loaded.")
    
    # def visualize_data(self, column):
    #     """Visualizes a column of the dataset."""
    #     if self.data is not None:
    #         import matplotlib.pyplot as plt
    #         plt.hist(self.data[column].dropna(), bins=30)
    #         plt.title(f"Distribution of {column}")
    #         plt.show()
    #     else:
    #         print("Data not loaded.")

# Example usage:
if __name__ == "__main__":
       
    # Create an instance of the DataAnalytics class
    recepeData = rawRecipe()
    
    # Load data
    recepeData.load_data()
    
    # Get data summary
    recepeData.summarize_dataset()
    
    # Check for missing values
    # analysis.check_null_values()
    
    # Visualize a specific column (e.g., 'age')
    # analysis.visualize_data('age')

Data loaded successfully. Shape: (231637, 12)
Data summary: Time ( minutes )
count    2.316370e+05
mean     9.398546e+03
std      4.461963e+06
min      0.000000e+00
25%      2.000000e+01
50%      4.000000e+01
75%      6.500000e+01
max      2.147484e+09
Name: minutes, dtype: float64
Data summary: Number of Steps
count    231637.000000
mean          9.765499
std           5.995128
min           0.000000
25%           6.000000
50%           9.000000
75%          12.000000
max         145.000000
Name: n_steps, dtype: float64
Data summary: Required Ingrediants
count    231637.000000
mean          9.051153
std           3.734796
min           1.000000
25%           6.000000
50%           9.000000
75%          11.000000
max          43.000000
Name: n_ingredients, dtype: float64
